In [1]:
from loaders.cognicity_loader import CognicityLoader
from learners.perceptron_learner import PerceptronLearner
from learners.svm_learner import SvmLearner

# import chennai_config
# config = chennai_config.config

# import jakarta_config
# config = jakarta_config.config

import jakarta_relabeled_config
config = jakarta_relabeled_config.config
RERUN = True

from image_recognition.goog_recog import GoogleLabeler
goog_learner = SvmLearner(config, CognicityLoader, GoogleLabeler)

from image_recognition.aws_recog import AwsLabeler
aws_learner = SvmLearner(config, CognicityLoader, AwsLabeler)

2019-08-21 06:53:38,057 - DEBUG - CognicityLoader constructed
2019-08-21 06:53:38,058 - DEBUG - GoogleLabeler constructed
2019-08-21 06:53:38,059 - DEBUG - CognicityLoader constructed
2019-08-21 06:53:38,059 - DEBUG - GoogleLabeler constructed
2019-08-21 06:53:38,096 - DEBUG - CognicityLoader constructed
2019-08-21 06:53:38,097 - DEBUG - AwsLabeler constructed
2019-08-21 06:53:38,097 - DEBUG - CognicityLoader constructed
2019-08-21 06:53:38,098 - DEBUG - AwsLabeler constructed


In [2]:
import random
validation_set = set(random.sample(config["flood_pkeys"], 10))
validation_set = validation_set.union(set(random.sample(config["no_flood_pkeys"], 10)))
len(validation_set)

20

In [3]:
th, th0 = goog_learner.run_learner("goog_separator.p", rerun=True, validation_keys=validation_set, params={"T":1000, "print":True})


2019-08-21 06:53:40,449 - INFO - Num Correct 15 Out of 20
2019-08-21 06:53:40,450 - INFO - Val score: 0.75


In [4]:
th, th0 = aws_learner.run_learner("aws_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":1000, "print":True})


2019-08-21 06:53:41,042 - DEBUG - logging from: default_jakarta_data/aws_labels_default.p
2019-08-21 06:53:47,026 - INFO - Num Correct 18 Out of 20
2019-08-21 06:53:47,027 - INFO - Val score: 0.9


In [5]:
from nlp.bow_labeler import BowLabeler
bow_learner = SvmLearner(config, CognicityLoader, BowLabeler)
th, th0 = bow_learner.run_learner("bow_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":400, "print":True})

2019-08-21 06:53:48,393 - DEBUG - CognicityLoader constructed
2019-08-21 06:53:48,394 - DEBUG - CognicityLoader constructed
2019-08-21 06:53:56,402 - INFO - Num Correct 17 Out of 20
2019-08-21 06:53:56,403 - INFO - Val score: 0.85


In [6]:
def _fill_no_data_spots(self, data_list, label_list):
    """
    Places zeros where data is missing
    Args:
        data_list: list of ndarrays where the first row is
            pkeys and the second row is a float datapoint
        labels: ndarray of labels
    Returns:
        pd.DataFrame:
    """
    all_pkeys = set()
    for each in data_list:
        all_pkeys.update(each[0,:].tolist())
        
    res = pd.DataFrame(all_pkeys,
                       columns=["pkey"]).set_index("pkey",
                                                   drop=True).sort_index()
    last_index = 0
    lab_df = pd.DataFrame(all_pkeys,
                       columns=["pkey"]).set_index("pkey",
                                                   drop=True).sort_index()
    lab_df["label"] = 0
    for i, (data, labels) in enumerate(zip(data_list, label_list)):
        # add_df = pd.DataFrame(data.T).set_index(0).rename(mapper={0: "pkey"}, axis="index")
        vect_len = data.shape[0]-1
        # pkeys should be index in the first row of data
        ind = pd.Index(data[0,:], name="pkey")
        add_df = pd.DataFrame(data=data[1:, :].T, index=ind, columns=range(last_index, last_index+vect_len)) # .set_index(0).rename_axis("pkey")
        last_index += vect_len
        print(add_df)
        # res = res.join(add_df).fillna(0)
        res = pd.concat([res, add_df], axis=1).fillna(0)
        l = pd.DataFrame(data=labels.T, columns=["label"], index=ind)
        lab_df.update(l)
        print(lab_df)
        # res = res.join(l)
        # res.loc[ind, "label"] = labels.T
                              
    return res.join(lab_df)


In [7]:
import numpy as np
import pandas as pd
a = np.array([[1, 2, 4, 65, 75],
         [113, 222, 4444, 65455, 7777]])
labs = np.array([[1, 1, 1, -1, -1]])

m = np.array([[1, 2, 4, 65, 70, 75],
              [5, 5, 5, 5,  5,  5],
              [5, 5, 5, 5,  5,  5]])
m_labs = np.array([[1, 1, 1, -1, -1, -1]])

b = np.array([[1, 2, 4, 65, 70, 75]])

c = np.vstack((b, np.zeros((1, b.shape[1]))))

b_df = pd.Index(b.T)

c_df = pd.DataFrame(c.T, columns=["pkey", "data"]).set_index("pkey", drop=False)

a_df = pd.DataFrame(a.T, columns=["pkey", "data"] )
# a_df.rename_axis( "pkey")
a_df.set_index("pkey", inplace=True, drop=False)
a_df

# c_df.loc[a_df.index] = a_df["data"]
c_df.loc[a_df.index] = a_df.loc[a_df.index]
eh = c_df.to_numpy()
eh[1:, :]
foo = _fill_no_data_spots(jakarta_config, [a], [labs])
foo = _fill_no_data_spots(jakarta_config, [a, a, m], [labs, labs, m_labs])
print(foo)

# data_list = [a, m]
# label_list = [labs, m_labs]
# 
# all_pkeys = set()
# for each in data_list:
#     all_pkeys.update(each[0,:].tolist())
# res = pd.DataFrame(all_pkeys,
#                    columns=["pkey"]).set_index("pkey",
#                                                drop=True).sort_index()
# last_index = 0
# lab_df = pd.DataFrame(all_pkeys,
#                    columns=["pkey"]).set_index("pkey",
#                                                drop=True).sort_index()
# lab_df["label"] = 0
# for i, (data, labels) in enumerate(zip(data_list, label_list)):
#     # add_df = pd.DataFrame(data.T).set_index(0).rename(mapper={0: "pkey"}, axis="index")
#     vect_len = data.shape[0]-1
#     # pkeys should be index in the first row of data
#     ind = pd.Index(data[0,:], name="pkey")
#     add_df = pd.DataFrame(data=data[1:, :].T, index=ind, columns=range(last_index, last_index+vect_len)) # .set_index(0).rename_axis("pkey")
#     last_index += vect_len
#     print(add_df)
#     # res = res.join(add_df).fillna(0)
#     res = pd.concat([res, add_df], axis=1).fillna(0)
#     l = pd.DataFrame(data=labels.T, columns=["label"], index=ind)
#     lab_df.update(l)
#     print(lab_df)
#     # res = res.join(l)
#     # res.loc[ind, "label"] = labels.T
#                           
# res

NameError: name 'jakarta_config' is not defined

In [ ]:
foo.to_numpy()

In [10]:
print("COMPARE WITH SVM: ")
learners = [ aws_learner, bow_learner]
t_data = []
t_labels = []
val_data = []
val_labels  = []
for each in learners:
    t_data.append(each.t_data_w_pkey)
    t_labels.append(each.t_labels)
    
    val_data.append(each.val_data_w_pkey)
    val_labels.append(each.val_labels)
    
df = _fill_no_data_spots("NOP", t_data, t_labels)
t_data = df.to_numpy()
t_labels = df["label"].to_numpy()

val_df = _fill_no_data_spots("NOP", val_data, val_labels)
val_data = val_df.to_numpy()
val_labels = val_df["label"].to_numpy()



COMPARE WITH SVM: 
         0     1     2          3          4     5     6          7     \
pkey                                                                     
1.0       0.0   0.0   0.0   0.000000  68.880173   0.0   0.0   0.000000   
3.0       0.0   0.0   0.0  23.932549  39.753395   0.0   0.0   0.000000   
9.0       0.0   0.0   0.0  53.905800   0.000000   0.0   0.0   0.000000   
10.0      0.0   0.0   0.0  31.867580   0.000000   0.0   0.0   0.000000   
20.0      0.0   0.0   0.0  98.371902   0.000000   0.0   0.0   0.000000   
23.0      0.0   0.0   0.0   0.000000   0.000000   0.0   0.0   0.000000   
101.0     0.0   0.0   0.0  34.086216   0.000000   0.0   0.0   0.000000   
116.0     0.0   0.0   0.0   0.000000   0.000000   0.0   0.0  25.766661   
130.0     0.0   0.0   0.0   0.000000   0.000000   0.0   0.0   0.000000   
132.0     0.0   0.0   0.0   0.000000   0.000000   0.0   0.0   0.000000   
148.0     0.0   0.0   0.0  63.691959   0.000000   0.0   0.0  32.084763   
160.0     0.0   0.0

         0          1     2          3          4     5     6          7     \
pkey                                                                          
4.0       0.0   0.000000   0.0   0.000000  76.289795   0.0   0.0   0.000000   
24.0      0.0   0.000000   0.0  80.680870   0.000000   0.0   0.0   0.000000   
382.0     0.0   0.000000   0.0   0.000000   0.000000   0.0   0.0   0.000000   
523.0     0.0   0.000000   0.0   0.000000   0.000000   0.0   0.0   0.000000   
562.0     0.0  18.483263   0.0   0.000000   0.000000   0.0   0.0   0.000000   
705.0     0.0   0.000000   0.0  95.619514   0.000000   0.0   0.0   0.000000   
744.0     0.0   0.000000   0.0   0.000000   0.000000   0.0   0.0   0.000000   
783.0     0.0   0.000000   0.0   0.000000   0.000000   0.0   0.0   0.000000   
1950.0    0.0   0.000000   0.0   0.000000   0.000000   0.0   0.0  25.871838   
2306.0    0.0   0.000000   0.0  28.334190   0.000000   0.0   0.0   0.000000   
2469.0    0.0   0.000000   0.0  23.832699   0.000000

In [11]:
t_labels.shape
t_data.shape

(810, 9916)

In [12]:
from sklearn import svm
clf = svm.SVC(gamma="scale", kernel="rbf", degree=3)
        # sklearn expects rows to be data points, we've gone with columns
clf.fit(t_data, t_labels)



SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [13]:
pred = clf.predict(val_data)

correct = np.sum(val_labels == pred)

total = val_data.shape[0]
percent_correct = correct/total
print("Num Correct " + str(correct) +
                 " Out of " + str(total))
print("Val score: " + str(percent_correct))
print("____________")

Num Correct 55 Out of 80
Val score: 0.6875
____________


In [14]:
from learners.ensemble_learner import EnsembleLearner
from learners.identity_learner import IdentityLearner
from flood_depth.flood_labeler import IdentityLabeler

from nlp.bow_labeler import BowLabeler
import random


results = dict()
for i in range(6, 11):
    validation_set = set(random.sample(config["flood_pkeys"], 40))
    validation_set = validation_set.union(set(random.sample(config["no_flood_pkeys"], 40)))
    
    aws_learner = SvmLearner(config, CognicityLoader, AwsLabeler)
    th, th0 = aws_learner.run_learner("aws_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":1000, "print":True})
    bow_learner = SvmLearner(config, CognicityLoader, BowLabeler)
    th, th0 = bow_learner.run_learner("bow_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":400, "print":True})
    
    fh_learner = IdentityLearner(config, CognicityLoader, IdentityLabeler)
    meh = fh_learner.run_learner("iden_separator.p", rerun=RERUN, validation_keys=validation_set, params={"T":400, "print":True})
    
    learners = [ aws_learner, bow_learner, fh_learner]
    names = ["aws.p", "bow.p", "fh.p"]
    en_learner = EnsembleLearner(config, names, learners)
    
    nn_model = en_learner.run_learner("jakarta_en.p", rerun=RERUN, validation_keys=validation_set, params={"hidden":i, "print":True, "T":2000})
    import math
    probs = math.e**en_learner.res
    probs
    import numpy as np
    p = probs.data.numpy()
    predicted = np.argmax(p, axis=1)
    val_labs = en_learner.val_labels
    true = np.where(val_labs <0, 0, 1)
    score = np.sum(predicted == true)/(true.shape[0])
    results[i] = (score, validation_set, en_learner, val_labs)
    print(score)
    print("COMPARE WITH SVM: ")
    t_data = en_learner.train_matrix[1:-1, :]
    t_labels = en_learner.train_matrix[-1, :]
    val_data = en_learner.val_matrix[1:-1, :]
    val_labels  = en_learner.val_matrix[-1, :]
    
    from sklearn import svm
    clf = svm.SVC(gamma="scale", kernel="rbf", degree=3)
            # sklearn expects rows to be data points, we've gone with columns
    clf.fit(t_data.T, t_labels)
    
    pred = clf.predict(val_data.T)
    
    correct = np.sum(val_labels == pred)
    
    total = val_data.shape[1]
    percent_correct = correct/total
    print("Num Correct " + str(correct) +
                     " Out of " + str(total))
    print("Val score: " + str(percent_correct))
    print("____________")
    

print(results)

2019-08-21 06:58:49,089 - DEBUG - CognicityLoader constructed
2019-08-21 06:58:49,090 - DEBUG - AwsLabeler constructed
2019-08-21 06:58:49,090 - DEBUG - CognicityLoader constructed
2019-08-21 06:58:49,090 - DEBUG - AwsLabeler constructed
2019-08-21 06:58:49,091 - DEBUG - logging from: default_jakarta_data/aws_labels_default.p
2019-08-21 06:58:55,289 - INFO - Num Correct 63 Out of 80
2019-08-21 06:58:55,290 - INFO - Val score: 0.7875
2019-08-21 06:58:56,597 - DEBUG - CognicityLoader constructed
2019-08-21 06:58:56,597 - DEBUG - CognicityLoader constructed
2019-08-21 06:59:04,703 - INFO - Num Correct 55 Out of 80
2019-08-21 06:59:04,704 - INFO - Val score: 0.6875
2019-08-21 06:59:08,416 - DEBUG - CognicityLoader constructed
2019-08-21 06:59:08,417 - DEBUG - IdentityLabeler constructed
2019-08-21 06:59:08,417 - DEBUG - CognicityLoader constructed
2019-08-21 06:59:08,418 - DEBUG - IdentityLabeler constructed
2019-08-21 06:59:09,994 - DEBUG - logging from: default_jakarta_data/aws.p
2019-08

                0
pkey             
1.0     -1.660653
3.0     -1.118228
4.0     -1.000004
10.0    -1.000261
20.0    -1.176689
23.0    -1.000087
24.0    -1.000093
101.0   -1.000269
116.0   -0.299538
130.0   -1.000058
132.0   -1.153772
148.0   -0.172845
160.0   -1.240941
162.0   -1.605714
163.0   -1.292670
168.0   -1.839818
208.0   -1.000433
209.0   -1.446723
210.0   -1.114649
211.0   -0.999884
212.0   -0.999971
213.0   -1.000236
316.0   -1.246267
331.0   -0.999995
360.0   -1.000011
363.0   -1.013159
368.0   -0.999857
382.0   -0.515882
384.0   -0.958683
394.0   -0.999759
...           ...
35651.0 -0.866661
35667.0 -0.896917
35691.0  0.088920
35703.0 -1.051957
35706.0 -0.465991
35739.0 -0.999688
35753.0  0.037096
35761.0  0.107302
35864.0 -1.000072
35880.0 -0.999810
35991.0  0.553006
36015.0 -0.570834
36016.0 -0.278670
36021.0 -0.926549
36022.0 -0.999885
36030.0 -1.000334
36135.0 -1.091135
36159.0 -1.000117
36162.0 -1.146861
36163.0 -1.000198
36231.0 -1.627977
36256.0 -0.075397
36257.0 -0

Loss: tensor(0.5796, grad_fn=<NllLossBackward>)
Loss: tensor(0.5794, grad_fn=<NllLossBackward>)
Loss: tensor(0.5792, grad_fn=<NllLossBackward>)
Loss: tensor(0.5791, grad_fn=<NllLossBackward>)
Loss: tensor(0.5789, grad_fn=<NllLossBackward>)
Loss: tensor(0.5787, grad_fn=<NllLossBackward>)
Loss: tensor(0.5786, grad_fn=<NllLossBackward>)
Loss: tensor(0.5784, grad_fn=<NllLossBackward>)
Loss: tensor(0.5782, grad_fn=<NllLossBackward>)
Loss: tensor(0.5780, grad_fn=<NllLossBackward>)
Loss: tensor(0.5779, grad_fn=<NllLossBackward>)
Loss: tensor(0.5777, grad_fn=<NllLossBackward>)
Loss: tensor(0.5775, grad_fn=<NllLossBackward>)
Loss: tensor(0.5773, grad_fn=<NllLossBackward>)
Loss: tensor(0.5772, grad_fn=<NllLossBackward>)
Loss: tensor(0.5770, grad_fn=<NllLossBackward>)
Loss: tensor(0.5768, grad_fn=<NllLossBackward>)
Loss: tensor(0.5767, grad_fn=<NllLossBackward>)
Loss: tensor(0.5765, grad_fn=<NllLossBackward>)
Loss: tensor(0.5763, grad_fn=<NllLossBackward>)
Loss: tensor(0.5762, grad_fn=<NllLossBac

Loss: tensor(0.5326, grad_fn=<NllLossBackward>)
Loss: tensor(0.5324, grad_fn=<NllLossBackward>)
Loss: tensor(0.5322, grad_fn=<NllLossBackward>)
Loss: tensor(0.5321, grad_fn=<NllLossBackward>)
Loss: tensor(0.5319, grad_fn=<NllLossBackward>)
Loss: tensor(0.5318, grad_fn=<NllLossBackward>)
Loss: tensor(0.5316, grad_fn=<NllLossBackward>)
Loss: tensor(0.5315, grad_fn=<NllLossBackward>)
Loss: tensor(0.5313, grad_fn=<NllLossBackward>)
Loss: tensor(0.5312, grad_fn=<NllLossBackward>)
Loss: tensor(0.5311, grad_fn=<NllLossBackward>)
Loss: tensor(0.5309, grad_fn=<NllLossBackward>)
Loss: tensor(0.5308, grad_fn=<NllLossBackward>)
Loss: tensor(0.5306, grad_fn=<NllLossBackward>)
Loss: tensor(0.5305, grad_fn=<NllLossBackward>)
Loss: tensor(0.5304, grad_fn=<NllLossBackward>)
Loss: tensor(0.5302, grad_fn=<NllLossBackward>)
Loss: tensor(0.5301, grad_fn=<NllLossBackward>)
Loss: tensor(0.5300, grad_fn=<NllLossBackward>)
Loss: tensor(0.5299, grad_fn=<NllLossBackward>)
Loss: tensor(0.5297, grad_fn=<NllLossBac

Loss: tensor(0.5013, grad_fn=<NllLossBackward>)
Loss: tensor(0.5012, grad_fn=<NllLossBackward>)
Loss: tensor(0.5012, grad_fn=<NllLossBackward>)
Loss: tensor(0.5011, grad_fn=<NllLossBackward>)
Loss: tensor(0.5010, grad_fn=<NllLossBackward>)
Loss: tensor(0.5009, grad_fn=<NllLossBackward>)
Loss: tensor(0.5008, grad_fn=<NllLossBackward>)
Loss: tensor(0.5008, grad_fn=<NllLossBackward>)
Loss: tensor(0.5007, grad_fn=<NllLossBackward>)
Loss: tensor(0.5006, grad_fn=<NllLossBackward>)
Loss: tensor(0.5005, grad_fn=<NllLossBackward>)
Loss: tensor(0.5004, grad_fn=<NllLossBackward>)
Loss: tensor(0.5004, grad_fn=<NllLossBackward>)
Loss: tensor(0.5003, grad_fn=<NllLossBackward>)
Loss: tensor(0.5002, grad_fn=<NllLossBackward>)
Loss: tensor(0.5001, grad_fn=<NllLossBackward>)
Loss: tensor(0.5001, grad_fn=<NllLossBackward>)
Loss: tensor(0.5000, grad_fn=<NllLossBackward>)
Loss: tensor(0.4999, grad_fn=<NllLossBackward>)
Loss: tensor(0.4998, grad_fn=<NllLossBackward>)
Loss: tensor(0.4997, grad_fn=<NllLossBac

...

Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBackward>)
Loss: tensor(0.1282, grad_fn=<NllLossBac

Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBackward>)
Loss: tensor(0.1274, grad_fn=<NllLossBac

Loss: tensor(0.1267, grad_fn=<NllLossBackward>)
Loss: tensor(0.1267, grad_fn=<NllLossBackward>)
Loss: tensor(0.1267, grad_fn=<NllLossBackward>)
Loss: tensor(0.1267, grad_fn=<NllLossBackward>)
Loss: tensor(0.1267, grad_fn=<NllLossBackward>)
Loss: tensor(0.1267, grad_fn=<NllLossBackward>)
Loss: tensor(0.1267, grad_fn=<NllLossBackward>)
Loss: tensor(0.1267, grad_fn=<NllLossBackward>)
Loss: tensor(0.1267, grad_fn=<NllLossBackward>)
Loss: tensor(0.1267, grad_fn=<NllLossBackward>)
Loss: tensor(0.1266, grad_fn=<NllLossBackward>)
Loss: tensor(0.1266, grad_fn=<NllLossBackward>)
Loss: tensor(0.1266, grad_fn=<NllLossBackward>)
Loss: tensor(0.1266, grad_fn=<NllLossBackward>)
Loss: tensor(0.1266, grad_fn=<NllLossBackward>)
Loss: tensor(0.1266, grad_fn=<NllLossBackward>)
Loss: tensor(0.1266, grad_fn=<NllLossBackward>)
Loss: tensor(0.1266, grad_fn=<NllLossBackward>)
Loss: tensor(0.1266, grad_fn=<NllLossBackward>)
Loss: tensor(0.1266, grad_fn=<NllLossBackward>)
Loss: tensor(0.1266, grad_fn=<NllLossBac

2019-08-21 07:01:41,692 - DEBUG - dumping model to: default_jakarta_data/jakarta_en.p


tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>)
Loss: tensor(0.1263, grad_fn=<NllLossBackward>

In [ ]:
l = validation_set
sorted(l)

In [ ]:
t_data = en_learner.train_matrix[1:-1, :]
t_labels = en_learner.train_matrix[-1, :]
val_data = en_learner.val_matrix[1:-1, :]
val_labels  = en_learner.val_matrix[-1, :]

from sklearn import svm
clf = svm.SVC(gamma="scale", kernel="rbf", degree=3)
        # sklearn expects rows to be data points, we've gone with columns
clf.fit(t_data.T, t_labels)

pred = clf.predict(val_data.T)

correct = np.sum(val_labels == pred)

total = val_data.shape[1]
percent_correct = correct/total
print("Num Correct " + str(correct) +
                 " Out of " + str(total))
print("Val score: " + str(percent_correct))
print(pred)
t_data[:,2:5]

In [ ]:
en_learner.val_matrix

In [ ]:
predicted

In [ ]:
import pytorch
a = pytorch.tensor(.4)

In [ ]:
predicted

validation_set

In [ ]:
predicted